In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import pyterrier as pt
if not pt.java.started():
    pt.java.init()

import os, sys
os.chdir('/mnt/primary/exposure-fairness-extend')


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [2]:
import config, fair_utils

msmarco-passage/dev documents: 100%|██████████| 8841823/8841823 [00:25<00:00, 340142.39it/s]


In [3]:
import importlib
importlib.reload(config)
importlib.reload(fair_utils)

msmarco-passage/dev documents: 100%|██████████| 8841823/8841823 [00:26<00:00, 337790.70it/s]


<module 'fair_utils' from '/mnt/primary/exposure-fairness-extend/fair_utils.py'>

In [4]:
csv_path = f'{config.data_dir}/bm25_{config.dataset_name}_{config.topics_name}_{config.retrieve_num}_{500}_{config.kmeans_vec[0]}.csv'
df = pd.read_csv(csv_path, index_col=0).reset_index()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/datasets/cxj/exposure-fairness-extend/bm25_msmarco-passage_dev_200_500_scikit_dense.csv'

In [25]:
import importlib
importlib.reload(config)
importlib.reload(fair_utils)

<module 'fair_utils' from '/mnt/primary/exposure-fairness-extend/fair_utils.py'>

In [46]:
csv = './all_models_gini_ndcg_map_cut20_from_100.csv'
df = pd.read_csv(csv,index_col=0).reset_index()
df

,a,b
0,10,30
1,20,40


In [47]:
df2 = pd.DataFrame({'a':[100,200], 'b':[300,400]})

In [48]:
df2

,a,b
0,100,300
1,200,400


In [49]:
df2.to_csv(csv, index=False)

In [38]:
csv = './all_models_gini_ndcg_map_cut20_from_100.csv'
df = pd.read_csv(csv,index_col=0).reset_index()


,a,b
0,1,3
1,2,4


In [41]:
df

,a,b
0,1,3
1,2,4


In [7]:
res = 'tctcolbert_msmarco-passage_dev_200.res'
df = fair_utils.convert_res2df(f'{config.data_dir}/{res}')

converting /mnt/datasets/cxj/exposure-fairness-extend/tctcolbert_msmarco-passage_dev_200.res into a dataframe
done


In [ ]:
df.head(10)


In [26]:
qrels_res = 'qrels_dev.res'
res_file = f'bm25_msmarco-passage_dev_200_diver_df_lbda1.0_cut20.res'
result = fair_utils.cal_metrics(qrels_res, res_file)

['trec_eval', '-m', 'ndcg_cut.10', '-m', 'map', '-m', 'recip_rank', '-m', 'P.10', '/mnt/datasets/cxj/exposure-fairness-extend/qrels_dev.res', '/mnt/datasets/cxj/exposure-fairness-extend/bm25_msmarco-passage_dev_200_diver_df_lbda1.0_cut20.res']
{}


In [9]:
top_100_df = df.groupby("qid", group_keys=False).head(100)

In [13]:
top_100_df.head(210)

,qid,Q0,docid,rank,score,run,docno,query
0,1048578,Q0,7187234,0,81.498650,tctcolbert_msmarco-passage_dev_200,7187234,cost of endless pools swim spa
1,1048578,Q0,4567130,1,81.423740,tctcolbert_msmarco-passage_dev_200,4567130,cost of endless pools swim spa
2,1048578,Q0,7187239,2,80.703580,tctcolbert_msmarco-passage_dev_200,7187239,cost of endless pools swim spa
3,1048578,Q0,2078221,3,80.142410,tctcolbert_msmarco-passage_dev_200,2078221,cost of endless pools swim spa
4,1048578,Q0,5365328,4,79.947495,tctcolbert_msmarco-passage_dev_200,5365328,cost of endless pools swim spa
5,1048578,Q0,7187235,5,79.797640,tctcolbert_msmarco-passage_dev_200,7187235,cost of endless pools swim spa
6,1048578,Q0,5365326,6,79.752335,tctcolbert_msmarco-passage_dev_200,5365326,cost of endless pools swim spa
7,1048578,Q0,7187241,7,79.738750,tctcolbert_msmarco-passage_dev_200,7187241,cost of endless pools swim spa
8,1048578,Q0,4332303,8,79.635760,tctcolbert_msmarco-passage_dev_200,4332303,cost of endless pools swim spa
9,1048578,Q0,4332300,9,79.604480,tctcolbert_msmarco-passage_dev_200,4332300,cost of endless pools swim spa


In [ ]:
fair_utils.save_trec_res(df, f'{config.data_dir}/tctcolbert_msmarco-passage_dev_200_cut100.res','tctcolbert_msmarco-passage_dev_200_cut100')

In [ ]:
raw_200 = fair_utils.convert_res2df(f'{config.data_dir}/tctcolbert_msmarco-passage_dev_200.res')

In [ ]:
raw_100_csv = 'tctcolbert_msmarco-passage_dev_200_cut100.csv'
raw_100 = pd.read_csv(f'{config.data_dir}/{raw_100_csv}', index_col=0).reset_index()

In [ ]:
divers_200_csv = 'tctcolbert_msmarco-passage_dev_200_diver_df.csv'
divers_200 = pd.read_csv(f'{config.data_dir}/{divers_200_csv}', index_col=0).reset_index()

In [ ]:
divers_100_csv = 'tctcolbert_msmarco-passage_dev_200_diver_df_cut100.csv'
divers_100 = pd.read_csv(f'{config.data_dir}/{divers_100_csv}', index_col=0).reset_index()

In [ ]:
raw_200.head(201)

In [ ]:
divers_200.head(201)

In [73]:
sub_raw = raw_100[raw_100['qid'] == 1000].copy()

In [74]:
sub_divers = divers_100[divers_100['qid'] == 1000].copy()

In [90]:
sub_raw['docno2'] = sub_divers['docno'].to_list()
sub_raw['rank2'] = sub_divers['rank'].to_list()
sub_raw['score2'] = sub_divers['score'].to_list()
sub_raw = sub_raw[['qid', 'docno','docno2', 'rank', 'rank2', 'score', 'score2']]

In [91]:
sub_raw[sub_raw['docno'] != sub_raw['docno2']].shape

(94, 7)

In [92]:
sub_raw[sub_raw['docno'] == sub_raw['docno2']]

,qid,docno,docno2,rank,rank2,score,score2
45000,1000,7391782,7391782,0,0,80.015340,-0.0
45001,1000,3486882,3486882,1,1,78.794624,-1.0
45002,1000,5317671,5317671,2,2,78.775860,-2.0
45033,1000,8772941,8772941,33,33,78.153160,-33.0
45047,1000,567659,567659,47,47,78.060010,-47.0
45074,1000,4398353,4398353,74,74,77.892050,-74.0


In [93]:
sub_raw[sub_raw['docno'] != sub_raw['docno2']]

,qid,docno,docno2,rank,rank2,score,score2
45003,1000,3486885,2715343,3,3,78.674790,-3.0
45004,1000,2715343,5527654,4,4,78.610460,-4.0
45005,1000,3486881,7391786,5,5,78.592766,-5.0
45006,1000,7391786,7391789,6,6,78.591450,-6.0
45007,1000,5527654,5918288,7,7,78.553314,-7.0
45008,1000,7391789,3486885,8,8,78.508804,-8.0
45009,1000,7391788,1652454,9,9,78.400055,-9.0
45010,1000,1652454,7391784,10,10,78.391030,-10.0
45011,1000,7391783,3486881,11,11,78.351010,-11.0
45012,1000,5918288,3963414,12,12,78.344640,-12.0


In [ ]:
csv_file = 'tctcolbert_msmarco-passage_dev_200_diver_df_cut100.res'
csv_file_df = fair_utils.convert_res2df(f'{config.data_dir}/{csv_file}')

In [ ]:
csv_file_df.head()

In [ ]:
df.head()

In [ ]:
result_res_file = f'{config.data_dir}/tctcolbert_msmarco-passage_dev_200_diver_df.res'
fair_utils.save_trec_res(diver_df, result_res_file, "tctcolbert_msmarco-passage_dev_200_diver_df")

In [ ]:
np.zeros(2)

In [ ]:
df = pd.DataFrame([['1','abc']], columns=['docno','text'])

In [ ]:
csv='./testcxjdf.csv'
df.to_csv(csv, index=False)

In [ ]:
csv_measures='./testcxjdf_measures.csv'

In [ ]:
df2 = pd.read_csv(csv_measures, index_col=0).reset_index()
df2.to_csv(csv_measures, index=False)

In [ ]:
df2 = pd.read_csv(csv, index_col=0).reset_index()


In [ ]:
df

In [ ]:
result_csv = "/mnt/datasets/cxj/exposure-fairness/v2/bm25_msmarco-passage_dev_100.csv"
df = pd.read_csv(result_csv, index_col=0).reset_index()

In [ ]:
df

In [ ]:
trec_file = "tctcolbert_msmarco-passage_dev_200.res"

In [ ]:
print(f'{config.data_dir}/{os.path.splitext(trec_file)[0]}_diver_df.csv')